# Requester's Guide 

The current release of Ravenverse provides a collection of easy to use libraries that allow requesters to build mathematical algorithms or models and compute these operations by distributing them across multiple clients. This provides an increase in speed and efficiency when dealing with a large number of mathematical operations.

This tutorial explains how to build a distributed graph as well as federated analytics graph that the requester can compile and execute via the nodes present on the Ravenverse network. 

### Installing Dependencies

Ravenverse provides Requesters with a library called RavOp which contains a large number of mathematical operations on mathematical objects like scalars, vectors, matrices and tensors.

In [ ]:
!pip install ravop

In [ ]:
%env TOKEN=abcdefg
%env RAVENVERSE_URL=http://0.0.0.0:8081
%env RAVENVERSE_FTP_HOST=0.0.0.0
%env RAVENVERSE_FTP_URL=0.0.0.0

### Initializing Ravop with Ravenverse Token

The requester must connect to the Ravenverse using a unique token that they can generate by logging into Raven's Website using their MetaMask wallet credentials.

In [2]:
import ravop as R

R.initialize(ravenverse_token='<TOKEN>')

2023-03-22 13:20:55,208 [MainThread  ] [DEBUG]  Checking version of Ravop...
2023-03-22 13:20:55,641 [MainThread  ] [DEBUG]  Please update Ravop to the latest version.
2023-03-22 13:20:55,644 [MainThread  ] [DEBUG]  Initializing...
2023-03-22 13:20:55,646 [MainThread  ] [DEBUG]  Creating FTP developer credentials...
2023-03-22 13:21:09,116 [MainThread  ] [DEBUG]  FTP Upload Blocksize:409600
2023-03-22 13:21:09,119 [MainThread  ] [DEBUG]  FTP User credentials:dev.server.ravenverse.ai 4596477436 RIB15J3C0M
2023-03-22 13:21:09,305 [MainThread  ] [DEBUG]  Initialized Successfully!
2023-03-22 13:21:09,421 [MainThread  ] [DEBUG]  No Currently Active Graph! 


## Distributed Computing

Distributed Computing is the linking of various computing resources like PCs and smartphones to share and coordinate their processing power for a common computational requirement, such as the training of a large Machine Learning model. These resources or nodes communicate with a central server and in some cases with each other, such that each node receives some data and completes a subset of a task. These nodes can coordinate their computations to complete a large and complex computational requirement in a fast and efficient manner.

The following walkthrough demonstrates how a requester should build a graph, compile it and then execute their graph to get the results.

### Step 1 - Defining a Graph

In the Ravenverse, each script compiled by a requester is treated as a collection of RavOP Operations called Graph. 

***Note:*** In the current release, the requester can execute only 1 graph with their unique token. Therefore, to clear any previous/existing graphs, the requester must use R.flush() method.

In [3]:
R.flush()

R.Graph(name='test', algorithm='test_graph', approach='distributed')

2023-03-22 13:21:22,529 [MainThread  ] [DEBUG]  
2023-03-22 13:21:22,530 [MainThread  ] [DEBUG]  {'message': 'Requester flushed'}


The ```name``` and ```algorithm``` parameters can be set to any string. However, the ```approach``` needs to be set to either "distributed" or "federated".

### Step 2 - Creating Operations (Ops)

The RavOp documentation can be found [here](https://ravenprotocol.gitbook.io/ravenverse/ravop). It has a detailed overview of the various Operations available. 

In [4]:
a = R.t([1, 2, 3])
b = R.t([4, 5, 6])
c = a + b

### Step 3 - Making Ops Persist

Persisting Ops are a special category of Ops that stay in the ravenverse once the graph gets executed. The requester must explicitly mention which ops they want to save in their code. It is a good idea to write the code such that persisting ops contain the relevant results (in this case, variable - ```c```).

***Note:*** Make sure that the ```name``` parameter for each persisting Op is unique within a graph so that later it can be retrieved.

In [5]:
c.persist_op(name='c_output')

2023-03-22 13:21:36,359 [MainThread  ] [DEBUG]  
2023-03-22 13:21:36,362 [MainThread  ] [DEBUG]  Persisting Op: c_output


### Step 4 - Activate the Graph

Once all Ops of the graph have been defined, the requester must activate their graph. This step completes the compilation procedure and makes the graph ready for execution. No more Ops can be added to the graph after this.

In [6]:
R.activate()

2023-03-22 13:21:42,252 [MainThread  ] [DEBUG]  {'message': 'Ops Persisted Successfully!'}
2023-03-22 13:21:42,403 [MainThread  ] [DEBUG]  

2023-03-22 13:21:42,406 [MainThread  ] [DEBUG]  Graph Compiled Successfully. Ready to Execute!
2023-03-22 13:21:42,407 [MainThread  ] [DEBUG]  Cost: 0.5758 RAVEN TOKENS
2023-03-22 13:21:42,408 [MainThread  ] [DEBUG]  Max Participants: 1


'Graph Compiled Successfully. Ready to Execute!'

This displays the cost and maximum number of allowable providers that can participate in this graph. 

### Step 5 - Execute the Graph

On execution, the graph will be split into smaller subgraphs which will be distributed to the participating compute nodes in the network. The requester can also track the progress of the graph. Please note that the participants parameter must be set to a number less than the maximum number of allowable providers from the previous cell.

In [7]:
R.execute(participants=1)
R.track_progress()

2023-03-22 13:21:51,948 [MainThread  ] [DEBUG]  Graph Execute Initiated


Progress |████████████████████████████████████████| 100/100 [100%] in 5:46.1 (0.29/s) 

Graph Computed Successfully!


### Step 6 - Fetching Results

The Requester can now fetch the computed results of the Ops that they had previously set to persist.


In [7]:
output = R.fetch_persisting_op(op_name="c_output")
print(output)

[5, 7, 9]


## Distributed Deep Learning

RavDL is Raven Protocol's Distributed Deep Learning tool that allows requesters to easily build, train and test their neural networks by leveraging the compute power of participating nodes across the globe. More on this library can be found [here](https://github.com/ravenprotocol/ravdl). The repository also has sample deep learning model scripts for reference.

## Federated Analytics

Federated analytics is a new approach to data analysis in which key statistics like mean, variance, and standard deviation can be calculated across various private datasets without compromising privacy. It operates similarly to federated learning in that it runs local calculations over each client device’s data and only makes the aggregated findings — never any data from a specific device — available to requesters. Sensitive data like medical records, financial transactions, employee data, and others can be analyzed without leaving the premise.

The Ravop library now supports the creation of federated operations which requesters can leverage to conduct analysis without directly observing a client’s private data.

### Step 1 - Defining a Graph

Create a federated analytics graph by providing its name, approach, and rules to which clients must adhere.

- ```name```: The name for the graph set by the requester. Preferably a meaningful name that allows clients to identify the type of dataset desired by the requester.
- ```approach```: Set to ‘federated’.
- ```rules```: The rules dictionary must contain the names of all the columns of data required by the requester for aggregation and their corresponding constraints as shown above. The clients will then be able to filter their data accordingly. Note: An empty dictionary for a column signifies no constraints. All values in that column shall be considered.
- ```max_clients```: The number of clients whose data must be aggregated and returned to the requester.

***Note:*** In the current release, the requester can execute only 1 graph with their unique token. Therefore, to clear any previous/existing graphs, the requester must use R.flush() method.

In [10]:
import json

R.flush()
R.Graph(name="Office Data", approach="federated",
                rules=json.dumps({"rules": {"age": {"min": 18, "max": 80},
                                            "salary": {"min": 1, "max": 5},
                                            "bonus": {"min": 0, "max": 10},
                                            "fund": {}
                                            },
                                  "max_clients": 1}))

2022-06-27 23:56:15,913 [MainThread  ] [DEBUG]  
2022-06-27 23:56:15,914 [MainThread  ] [DEBUG]  {'message': 'Developer flushed'}


The results will be ready once ```max_clients``` number of clients have participated.

### Step 2 - Creation of Federated Ops

The following code snippet creates and adds ops to the previously declared graph.

In [11]:
mean = R.federated_mean()
variance = R.federated_variance()
standard_deviation = R.federated_standard_deviation()

### Step 3 - Making Ops Persist

Ops that have been set to persist can later be retrieved after execution of graph is complete.

***Note:*** Make sure that the ```name``` parameter for each persisting Op is unique within a graph so that later it can be retrieved.

In [12]:
mean.persist_op(name="mean")
variance.persist_op(name="variance")
standard_deviation.persist_op(name="standard_deviation")

2022-06-27 23:56:33,789 [MainThread  ] [DEBUG]  
2022-06-27 23:56:33,790 [MainThread  ] [DEBUG]  {'message': 'Op persist: mean'}
2022-06-27 23:56:33,801 [MainThread  ] [DEBUG]  
2022-06-27 23:56:33,802 [MainThread  ] [DEBUG]  {'message': 'Op persist: variance'}
2022-06-27 23:56:33,813 [MainThread  ] [DEBUG]  
2022-06-27 23:56:33,814 [MainThread  ] [DEBUG]  {'message': 'Op persist: standard_deviation'}


'Op persist: standard_deviation'

### Step 4 - Activate the Graph

Once all Ops of the graph have been defined, the requester must activate their graph. This step completes the compilation procedure and makes the graph ready for execution. No more Ops can be added to the graph after this.

In [13]:
R.activate()

2022-06-27 23:56:37,025 [MainThread  ] [DEBUG]  

2022-06-27 23:56:37,026 [MainThread  ] [DEBUG]  Graph Compiled Successfully. Ready to Execute!


'Graph Compiled Successfully. Ready to Execute!'

### Step 5 - Execute the Graph

On execution, the compute nodes will be allowed to participate if they possess the relevant data on their systems. The requester can also track the progress of the graph.

In [14]:
R.execute()
R.track_progress()

2022-06-27 23:56:42,478 [MainThread  ] [DEBUG]  Graph Execute Initiated
2022-06-27 23:56:42,479 [MainThread  ] [DEBUG]  


Progress |████████████████████████████████████████| 100/100 [100%] in 59.2s (1.69/s) 


2022-06-27 23:57:41,721 [MainThread  ] [DEBUG]  
Graph Computed Successfully!


### Step 6 - Fetching Results

The Requester can now fetch the computed results of the Ops that they had previously set to persist.


In [15]:
mean_output = R.fetch_persisting_op(op_name="mean")
print("mean: ", mean_output)

variance_output = R.fetch_persisting_op(op_name="variance")
print("variance: ", variance_output)

standard_deviation_output = R.fetch_persisting_op(op_name="standard_deviation")
print("standard_deviation: ", standard_deviation_output)

mean:  [{'age': 53.0}, {'bonus': 1.4444444444444444}, {'fund': 0.23333333333333334}, {'salary': 3.3333333333333326}]
variance:  [{'age': 96.0}, {'bonus': 0.011358024691358024}, {'fund': 0.004444444444444445}, {'salary': 0.08888888888888889}]
standard_deviation:  [{'age': 9.797958971132712}, {'bonus': 0.10657403385139376}, {'fund': 0.06666666666666668}, {'salary': 0.29814239699997197}]
